# Extraction et Analyse des Rues du Recensement de Lausanne en 1832 

## Imports

In [1]:
import pandas as pd
import numpy as np

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

RECENSEMENTS_PATH = './data/1832_v4.xlsx'
MAP_PATH = './data/nom_rues.csv'

## Chargement des données

In [2]:
recensements = pd.read_excel(RECENSEMENTS_PATH, sheet_name='Sheet1', usecols='B:V')
recensements.dtypes

nom_rue                       object
nom_rue_htr_corr              object
nom_rue_norm                  object
nom_rue_norm_2                object
no_maison                     object
proprietaire_nom              object
proprietaire_nom_corr         object
proprietaire_nom_htr_corr     object
proprietaire_nom_norm         object
proprietaire_nom_norm_2       object
proprietaire_nom_remarques    object
chef_prenom                   object
chef_prenom_htr_corr          object
chef_prenom_norm              object
chef_nom                      object
chef_nom_corr                 object
chef_nom_htr_corr             object
chef_nom_norm                 object
chef_nom_norm_2               object
chef_nom_remarques            object
chef_annee_naissance          object
dtype: object

In [3]:
recensements_rues = recensements['nom_rue_norm_2'].unique()

In [4]:
map_rues = pd.read_csv(MAP_PATH, sep=',')
map_rues = map_rues['nom_rue'].tolist()

In [5]:
def get_fuzzy_match(x, choices, scorer=fuzz.token_set_ratio, cutoff=0):
    result = process.extractOne(x, choices, scorer=scorer)
    if result and result[1] >= cutoff:
        return result[0]
    return None

recensements_rues = pd.DataFrame(recensements_rues, columns=['nom_rue_norm_2'])
recensements_rues['nom_rue_map'] = recensements_rues['nom_rue_norm_2'] \
    .apply(lambda x: get_fuzzy_match(x, map_rues, cutoff=70))
recencesments_rues = recensements_rues.set_index('nom_rue_norm_2')
recensements_rues.head(5)

,nom_rue_norm_2,nom_rue_map
0,marterey,None
1,bugnon,au bugnon
2,etraz,rue d etraz
3,villamont,None
4,mon repos,a mon repos


In [6]:
recensements_rues.to_csv('./data/recensements_rues.csv', index=False)

In [7]:
retrieved_rues_ratio = recensements_rues['nom_rue_map'].count() / recensements_rues['nom_rue_norm_2'].count()
print(f'There are {retrieved_rues_ratio:.2%} of the streets in the recensements that have a match in the map')

There are 25.83% of the streets in the recensements that have a match in the map


In [8]:
rues_count = recensements.groupby('nom_rue_norm_2')['nom_rue_norm_2'].count()
rues_count.head(10)

nom_rue_norm_2
a la tuerie ?      1
ale              186
barre            137
beau-soleil        1
beaulieu           4
beausite           1
bel-air            4
bellevue           4
bergieres          2
bethusy            1
Name: nom_rue_norm_2, dtype: int64

In [9]:
# add rues_count to recensements_rues
recensements_rues = recensements_rues \
    .join(rues_count, on='nom_rue_norm_2', how='left', rsuffix='_count') \
    .rename(columns={'nom_rue_norm_2_count': 'rue_count'})

In [10]:
recensements_rues.head()

,nom_rue_norm_2,nom_rue_map,rue_count
0,marterey,None,252
1,bugnon,au bugnon,4
2,etraz,rue d etraz,81
3,villamont,None,2
4,mon repos,a mon repos,1


In [11]:
mapped_rues = recensements_rues[recensements_rues['nom_rue_map'].notnull()]
mapped_rues_count = mapped_rues['rue_count'].sum()
mapped_rues_count

2558

In [13]:
total_rues_count = recensements_rues['rue_count'].sum()
total_rues_count

3674

In [15]:
print(f'There are {mapped_rues_count / total_rues_count:.2%} of the entries in the recensements that have a mapped street')

There are 69.62% of the entries in the recensements that have a mapped street


In [17]:
not_mapped_rues = recensements_rues[recensements_rues['nom_rue_map'].isnull()]
not_mapped_top_rues = not_mapped_rues.sort_values('rue_count', ascending=False).head(10)
not_mapped_top_rues

,nom_rue_norm_2,nom_rue_map,rue_count
0,marterey,None,252
47,ale,None,186
64,ouchy,None,123
86,cour,None,48
122,chailly,None,35
49,maupas,None,30
19,grotte,None,26
141,la sallaz,None,21
22,petit chene,None,18
142,vennes,None,17
